In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import anndata as ad
import os
from pathlib import Path

# QC

先成功复现质控的部分，抽样查看顺序是否一致，毕竟 R 和 Python 的处理可能不同

然后提取免疫细胞亚组，可以使用唯一标识符，比如 barcode？（如果唯一的话，可以用 unique 查看一下行数，否则可以自己生成一列 unique_id，省去转换对象的麻烦，不建议用绝对数字，最好是其他几列拼接成的字符串列，这样避免顺序不一致）

In [4]:
# 定义过滤阈值
nFeature_lower = 500
nFeature_upper = 10000
nCount_lower = 1000
nCount_upper = 100000
pMT_upper = 30
pHB_upper = 5

# 读取样本列表
samples_df = pd.read_excel("../data/metadata/patients_metadata.xlsx", usecols="A")
samples = samples_df['sample_id'].tolist()


In [5]:
# 读取所有样本数据
adatas = []
for i, sample in enumerate(samples):
    # 构建路径
    data_dir = Path(f"../data/cellranger/{sample}/filtered_feature_bc_matrix")
    
    # 读取 Cell Ranger 数据
    adata = sc.read_10x_mtx(
        data_dir,  # 矩阵目录
        var_names='gene_symbols',  # 使用基因符号作为变量名
        cache=True  # 缓存以提高读取速度
    )
    
    # 添加样本标识到观测名
    adata.obs_names = [f"{sample}_{x}" for x in adata.obs_names]
    
    # 添加样本信息到 obs
    adata.obs['sample'] = sample
    
    # 过滤低质量细胞（提前过滤空液滴）
    #sc.pp.filter_cells(adata, min_genes=nFeature_lower)  # 对应 min.cells=3
    
    adatas.append(adata)

# 合并所有样本
merged_adata = adatas[0].concatenate(
    adatas[1:],
    join='outer',  # 保留所有基因
    batch_key='sample',  # 存储样本来源的列名
    index_unique=None  # 不添加唯一索引
)

merged_adata

/tmp/ipykernel_816266/2487816240.py:26: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  merged_adata = adatas[0].concatenate(


AnnData object with n_obs × n_vars = 133736 × 33538
    obs: 'sample'
    var: 'gene_ids', 'feature_types'

In [11]:
with open("imm_ids.txt") as f:
    imm_ids = [line.strip() for line in f]

In [13]:
imm_subset = merged_adata[merged_adata.obs.index.isin(imm_ids), :]

In [ ]:
# 过滤低质量细胞（提前过滤空液滴）
sc.pp.filter_cells(merged_adata, min_genes=nFeature_lower)  # 对应 min.cells=3

# 计算质量控制指标
def calculate_qc(adata):
    # 线粒体基因（注意基因命名方式）
    adata.var['mt'] = adata.var_names.str.startswith('MT-')
    # 血红蛋白基因（同时匹配大小写）
    adata.var['hb'] = adata.var_names.str.upper().str.startswith(('HBA', 'HBB'))
    # 核糖体基因
    adata.var['ribo'] = adata.var_names.str.startswith(('RPS', 'RPL'))
    
    # 计算百分比
    sc.pp.calculate_qc_metrics(
        adata,
        qc_vars=['mt', 'hb', 'ribo'],
        percent_top=None,
        log1p=False,
        inplace=True
    )
    
    # 重命名列以匹配 R 代码
    adata.obs.rename(columns={
        'pct_counts_mt': 'pMT',
        'pct_counts_hb': 'pHB',
        'pct_counts_ribo': 'pRP'
    }, inplace=True)
    
    return adata

merged_adata = calculate_qc(merged_adata)

# 应用过滤条件
filtered_adata = merged_adata[
    (merged_adata.obs.n_genes_by_counts > nFeature_lower) &
    (merged_adata.obs.n_genes_by_counts < nFeature_upper) &
    (merged_adata.obs.total_counts > nCount_lower) &
    (merged_adata.obs.total_counts < nCount_upper) &
    (merged_adata.obs.pMT < pMT_upper) &
    (merged_adata.obs.pHB < pHB_upper), :
].copy()

filtered_adata

AnnData object with n_obs × n_vars = 114494 × 33538
    obs: 'sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pMT', 'total_counts_hb', 'pHB', 'total_counts_ribo', 'pRP'
    var: 'gene_ids', 'feature_types', 'mt', 'hb', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [22]:
imm_subset.write_h5ad('seurat_objects/imm_subset.h5ad')

/home/louhao/.conda/envs/info/lib/python3.10/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


# UCE Embedding

In [24]:
imm = sc.read_h5ad("seurat_objects/imm_subset.h5ad")
imm

AnnData object with n_obs × n_vars = 82888 × 33538
    obs: 'sample'
    var: 'gene_ids', 'feature_types'

略有出入，但是假装可以，然后继续先把代码搞定，scttransform 实在是太消耗时间了，其实应该把 immune 亚组挑选出来进行UCE 嵌入的

Final evaluated AnnData: dir/{dataset_name}.h5ad. This AnnData will be identical to the proccessed input anndata, but have UCE embeddings added in the .obsm["X_uce"] slot.


In [ ]:
# 激活环境，进入eval_single_anndata.py 文件所在路径，去除感叹号后复制到终端运行
# 参数解释详见 https://github.com/snap-stanford/uce
!CUDA_VISIBLE_DEVICES=1 python eval_single_anndata.py --adata_path /data/sjwlab/louhao/immune_polarization/code/seurat_objects/imm_subset.h5ad --dir /data/sjwlab/louhao/immune_polarization/code/UCE --species human --model_loc model_files/4layer_model.torch --batch_size 50

In [47]:
adata = sc.read_h5ad('seurat_objects/filtered_adata.h5ad')
adata

AnnData object with n_obs × n_vars = 114494 × 33538
    obs: 'sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pMT', 'total_counts_hb', 'pHB', 'total_counts_ribo', 'pRP'
    var: 'gene_ids', 'feature_types', 'mt', 'hb', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [ ]:
adata_uce = sc.read_h5ad('UCE/UCEfiltered_adata_uce_adata.h5ad')

In [ ]:
uce_emb = adata_uce.obsm['X_uce']
df_emb = pd.DataFrame(uce_emb, index=adata_uce.obs_names, columns=[f'uce_{i}' for i in range(1, uce_emb.shape[1]+1)])
df_emb.to_csv('UCE/uce_emb.csv.gz')